In [1]:

import sys
from pathlib import Path
notebook_path = Path().resolve()
sys.path.append(str(notebook_path.parent.parent))

In [2]:
sys.path


['c:\\Users\\PC\\Miniconda3\\envs\\causal_ml\\python311.zip',
 'c:\\Users\\PC\\Miniconda3\\envs\\causal_ml\\DLLs',
 'c:\\Users\\PC\\Miniconda3\\envs\\causal_ml\\Lib',
 'c:\\Users\\PC\\Miniconda3\\envs\\causal_ml',
 '',
 'c:\\Users\\PC\\Miniconda3\\envs\\causal_ml\\Lib\\site-packages',
 'c:\\Users\\PC\\Miniconda3\\envs\\causal_ml\\Lib\\site-packages\\win32',
 'c:\\Users\\PC\\Miniconda3\\envs\\causal_ml\\Lib\\site-packages\\win32\\lib',
 'c:\\Users\\PC\\Miniconda3\\envs\\causal_ml\\Lib\\site-packages\\Pythonwin',
 'D:\\repos\\cannibalization_reatail']

In [ ]:
# main.py

import sys
from pathlib import Path

notebook_path = Path().resolve()
sys.path.append(str(notebook_path.parent.parent))
import os
import pandas as pd
from typing import Optional
from src.conf import config
import src.preprocess_data.data_loader as dl
from src.preprocess_data.feature_engineering import build_full_panel

def save_panel(panel: pd.DataFrame, out_path: Optional[str] = None) -> str:
    """
    Guarda el panel en parquet (si pyarrow disponible) o CSV. Devuelve la ruta escrita.
    """
    if out_path is None:
        out_path = os.path.join(config.OUTPUT_DIR, "/panel_semanal.csv")

    try:
        # Preferimos parquet por tamaño/velocidad
        panel.to_parquet(out_path, index=False)
    except Exception:
        # Fallback a CSV si no hay pyarrow/fastparquet
        out_path = out_path.rsplit(".", 1)[0] + ".csv"
        panel.to_csv(out_path, index=False)
    return out_path

def main():
    # 1) Cargar datos crudos
    raw = dl.load_all_raw()

    # 2) Construir panel completo
    panel = build_full_panel(raw)

    # 3) Guardar y reportar
    out_file = save_panel(panel)
    print(f"Panel semanal listo → {out_file}")
    print(panel.info(memory_usage="deep"))
    print(panel.head(5))

if __name__ == "__main__":
    main()

D:\repos\cannibalization_reatail\src\utils\utils.py:13: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(full_path, **kwargs)
D:\repos\cannibalization_reatail\src\utils\utils.py:31: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  return s.fillna(False).astype(str).str.lower().isin(["true", "1", "t", "yes"]).astype("int8")
c:\Users\PC\Miniconda3\envs\causal_ml\Lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: divide by zero encountered in log1p
  result = getattr(ufunc, method)(*inputs, **kwargs)
c:\Users\PC\Miniconda3\envs\causal_ml\Lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: invalid value encountered in log1p
  result = getattr(ufunc, method)(*inputs, **kwargs)
